# Alexandra Koller
## Take Home Assignment
### Step 1: Data Transformation

In [110]:
# Step 1 loads patient enrollment data by month and year, identifies patient continuous enrollment spans,
# changes date formatting to a standard style, and saves the results as a patient enrollment span table. 

import pandas as pd

# Upload patient enrollment data
patient_month_year_path = r'/Users/alexandrakoller/Desktop/Koller_Take_Home_Assignment/patient_id_month_year.csv'
patient_month_year_df = pd.read_csv(patient_month_year_path)

# Convert patient enrollment dates from strings to dates
patient_month_year_df['month_year'] = pd.to_datetime(patient_month_year_df['month_year'], format='%m/%d/%Y')

# Sort data by patient id and enrollment month and year
patient_month_year_df = patient_month_year_df.sort_values(['patient_id', 'month_year'])

# For each patient ID, determine enrollment spans (for our purposes since enrollment is continuous, gaps would be greater than 1 month)
def group_enrollment_spans(group):
    gap = group['month_year'].diff().dt.days.gt(31).fillna(False)
    group['span_id'] = gap.cumsum()
    return group

patient_month_year_df = patient_month_year_df.groupby('patient_id', group_keys=False).apply(group_enrollment_spans)

# Determine each enrollment span start and end dates
patient_enrollment_span_df = patient_month_year_df.groupby(['patient_id', 'span_id']).agg(
    enrollment_start_date=('month_year', 'min'),
    enrollment_end_date=('month_year', 'max')
).reset_index()

# Since enrollment dates are inclusive, define enrollment_end_date as the last day of latest enrollment month
patient_enrollment_span_df['enrollment_end_date'] += pd.offsets.MonthEnd(0)

# Format enrollment start and end dates to standard formatting (YYYY-MM-DD)
patient_enrollment_span_df['enrollment_start_date'] = patient_enrollment_span_df['enrollment_start_date'].dt.strftime('%Y-%m-%d')
patient_enrollment_span_df['enrollment_end_date'] = patient_enrollment_span_df['enrollment_end_date'].dt.strftime('%Y-%m-%d')

# Exclude span_id column
patient_enrollment_span_df = patient_enrollment_span_df.drop(columns='span_id')

# Save enrollment result as a csv
patient_enrollment_span_path = r'/Users/alexandrakoller/Desktop/Koller_Take_Home_Assignment/patient_enrollment_span.csv'
patient_enrollment_span_df.to_csv(patient_enrollment_span_path, index=False)

# Answer 1: report the number of enrollment rows = 3105
number_rows = len(patient_enrollment_span_df)
print(f'Number of rows in patient_enrollment_span_df: {number_rows}')




Number of rows in patient_enrollment_span_df: 3105


### Step 2: Data Aggregation

In [113]:
# Step 2 loads outpatient visit data and enrollment spans,
# filters outpatient visits to only those within enrollment spans for each patient,
# calculates the total number of outpatient visits and distinct visit days per enrollment span per patient,
# and saves the findings as a results table.

import pandas as pd

# Upload outpatient visit data
outpatient_visits_file_path = r'/Users/alexandrakoller/Desktop/Koller_Take_Home_Assignment/outpatient_visits_file.csv'
outpatient_visits_file_df = pd.read_csv(outpatient_visits_file_path)

# Convert outpatient visit dates from strings to datetimes
outpatient_visits_file_df['date'] = pd.to_datetime(outpatient_visits_file_df['date'], format='%m/%d/%Y')

# Upload patient enrollment spans from Step 1
patient_enrollment_span_path = r'/Users/alexandrakoller/Desktop/Koller_Take_Home_Assignment/patient_enrollment_span.csv'
patient_enrollment_span_df = pd.read_csv(patient_enrollment_span_path)

# Convert enrollment start and end dates datetimes
patient_enrollment_span_df['enrollment_start_date'] = pd.to_datetime(patient_enrollment_span_df['enrollment_start_date'])
patient_enrollment_span_df['enrollment_end_date'] = pd.to_datetime(patient_enrollment_span_df['enrollment_end_date'])

# Count total outpatient visits and number of distinct visit days for each patient's enrollment spans
def aggregate_visits(row):
    patient_id = row['patient_id']
    start_date = row['enrollment_start_date']
    end_date = row['enrollment_end_date']
    
    # Filter outpatient visits within each patient's enrollment spans
    outpatient_visits = outpatient_visits_file_df[
        (outpatient_visits_file_df['patient_id'] == patient_id) &
        (outpatient_visits_file_df['date'] >= start_date) &
        (outpatient_visits_file_df['date'] <= end_date)
    ]
    
    visit_count = outpatient_visits['outpatient_visit_count'].sum()
    day_count = outpatient_visits['date'].nunique()
    
    return pd.Series([visit_count, day_count])

# Calculate totals for each enrollment span
patient_enrollment_span_df[['ct_outpatient_visits', 'ct_days_with_outpatient_visit']] = (
    patient_enrollment_span_df.apply(aggregate_visits, axis=1)
)

# Save visit and day count results as a csv
result_csv_path = r'/Users/alexandrakoller/Desktop/Koller_Take_Home_Assignment/result.csv'
patient_enrollment_span_df.to_csv(result_csv_path, index=False)

# Report the number of distinct values of ct_days_with_outpatient_visit in result.csv. 
distinct_values_count = patient_enrollment_span_df['ct_days_with_outpatient_visit'].nunique()
print(f'Number of distinct values in ct_days_with_outpatient_visit: {distinct_values_count}')


Number of distinct values in ct_days_with_outpatient_visit: 33
